# Case 2: Cargo Ship
The ship MS Leiden is traveling on a route involving five harbors in north-
ern Europe: Rotterdam, Hamburg, Kiel, Aarhus, and Copenhagen. You have
been asked to help the Captain with the loading and unloading plan (for the
destination harbors) that produces

- even and well-balanced solution
- a solution that is easy to unload (so that containers that are earlier un-loaded are not beneath containers that are later unloaded)
- solutions that can integrate as many containers as possible

The following details about the problem are provided:
1. The ship is now docked in Rotterdam. After visiting Rotterdam, the ship
will be traveling to Hamburg, Aarhus, and Copenhagen, in that order. The
containers destined for Rotterdam have been unloaded and the containers
destined for the remaining three harbors have to be loaded on the ship.
2. MS Leiden is a rather small container ship. It has eight bays, three tiers
and four rows. The layout of the ship is shown in the figure below.
3. Containers should be loaded such that a container that has to be unloaded
earlier should be placed in a higher position.
4. Each cell in the above figure is able to hold a forty-foot container. That
is, the ship has room for 8 × 3 × 4 = 96 forty-foot containers.
5. A forty-foot container can be placed on-top of another forty-foot container,
but not on top of an empty cell. We assume that only forty-foot containers
are loaded on the ship. Each container has a destination port.
6. Each container $i$ has a certain weight $w_i$. If a container is much heavier
than another $i$ container $j$, say $w_i − w_j > δ_w$, then it is not allowed to
place $w_i$ on top of $w_j$ .
7. The containers should be balanced in a way that is evenly distributed. The
longitudinal center of gravity should be as close as possible to the middle
of the container section of the ship. Secondly, the latitudinal center of
gravity should be as much to the middle as possible. Note that the center
of gravity can be computed as the weighted sum of the centroids of the
containers, where the weights are the total weight of the containers.

In [3]:
import desdeo
import numpy as np

# desdeo.probdefs.Problem.__abstractmethods__ = set()

![title](cargo1.png)

![title](cargo2.png)

![title](cargo3.png)

In [ ]:
order = ['Rotterdam', 'Hamburg', 'Aarhus', 'Copenhagen']

## Custom dataset

- Load has to be balanced in every part of the journey (Rotterdam -> Hamburg, Hamburg -> Aarhus, Aarhus -> Copenhagen)
- 

In [ ]:
# Different scenarios with different weights (ID, weight)
# The ID is the ID of the cargo and is between 1 and 10
# The weight is between 3750 and 27600 and random

# Scenario 1
Hamburg = 1
Aarhus = 2
Copenhagen = 3
Data = [(1, 5883), (1, 3485), (1, 9985), (1, 17600), 
        (2, 7352), (2, 9230), (2, 4699), (2, 16400), 
        (3, 14500), (3, 5773), (3, 12258)]

# Scenario 2 with different weights (ID, weight)


## Code

Balance score can be calculated as:
The containers should be balanced in a way that is evenly distributed. The
longitudinal center of gravity should be as close as possible to the middle
of the container section of the ship. Secondly, the latitudinal center of
gravity should be as much to the middle as possible. Note that the center
of gravity can be computed as the weighted sum of the centroids of the
containers, where the weights are the total weight of the containers.

In [9]:
def sum_tiers(configuration):

    only_weights = configuration[:, :, :, 1]
    summed_configuration = np.sum(only_weights, axis=1)

    return summed_configuration


def get_longitudinal_center(summed_configuration):
    """
    Parameters
    ----------
        configuration: list of integers
            Shape: (8,3,4,2)
            Where containers are placed in the ship.
    Returns
    -------
        center: float between 1 and 8
            The longitudinal center of the ship.
    """
    # Calculate the longitudinal center of the ship
    # The longitudinal center is the center of mass of the ship
    # The center of mass is the sum of the products of the weights and their positions
    # divided by the sum of the weights

    # Get the weights and the positions of the containers
    weights = summed_configuration.flatten()
    positions = np.arange(1, 9)

    # Calculate the center of mass
    center = np.sum(weights * positions) / np.sum(weights)

    return center


def get_lattitudinal_center(summed_configuration):
    """
    Parameters
    ----------
        configuration: list of integers
            Shape: (8,3,4,2)
            Where containers are placed in the ship.
    Returns
    -------
        center: float between 1 and 4
            The lattitudinal center of the ship.
    """
    # Calculate the lattitudinal center of the ship
    # The lattitudinal center is the center of mass of the ship
    # The center of mass is the sum of the products of the weights and their positions
    # divided by the sum of the weights

    # Get the weights and the positions of the containers
    weights = summed_configuration.flatten()
    positions = np.arange(1, 5)

    # Calculate the center of mass
    center = np.sum(weights * positions) / np.sum(weights)

    return center


def calculate_centers(configuration):

    summed_configuration = sum_tiers(configuration)

    longitudinal_center = get_longitudinal_center(summed_configuration)
    lattitudinal_center = get_lattitudinal_center(summed_configuration)

    print(longitudinal_center, lattitudinal_center)

    return longitudinal_center, lattitudinal_center

calculate_centers(np.ones((8, 3, 4, 2)))

ValueError: operands could not be broadcast together with shapes (32,) (8,) 